<a href="https://colab.research.google.com/github/ContextLab/psyc32-n-queens/blob/main/n-queens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The $n$-Queens puzzle: Introduction and overview

The classic [eight queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) refers to the challenge of placing eight [chess](https://en.wikipedia.org/wiki/Chess) [queens](https://en.wikipedia.org/wiki/Queen_(chess)) on an $8 \times 8$ board such that no queen is attacking any other.  Specifically, there can only be (at most) a single queen on any row, column, and diagonal of the board.  Here is one solution to the eight queens puzzle (there are 92 distinct solutions in total, or 12 if rotations and reflections are not counted as separate solutions):

![8-queens](https://i.stack.imgur.com/D32OV.png)

In principle, the eight queens puzzle may be generalized to imaginary chessboards of any size.  For $n \geq 1$, placing $n$ queens on an $n \times n$ chess board is referred to as the $n$-queens puzzle.

For this assignment, you'll be building a solver for the $n$-queens puzzle.  There are two general functions you'll need to write:
1. A function for printing out a board position (e.g., depicting the configuration of the pieces visually)
2. A function for checking whether a given position corresponds to a solution (i.e., whether any queens are attacking each other)

Using these two functions, you'll be provided with functions for returning all solutions (given $n \geq 1$) and for returning the *number* of solutions (given $n \geq 1$).

## Some history of the $n$-queens puzzle
In [a 2017 paper](https://www.jair.org/index.php/jair/article/view/11079), the $n$-queens puzzle (sometimes called the $n$-queens problem) was shown to be a member of a set of problems in computer science that are [NP-complete](https://en.wikipedia.org/wiki/NP-completeness).  Exploring or proving NP-completeness is beyond the scope of our course, but the super high-level intuition for NP-completeness can be summarized as:
  - For all known ways of solving them, NP-complete problems take a very long time to solve (their associated computations don't scale well)
  - Because all NP-complete problems can be efficiently(ish) converted to other NP-complete problems, solving any one NP-complete problem efficiently would also provide an efficient solution for *all* NP-complete problems.

Solving NP-complete problems efficiently is an [open question in computer science](https://en.wikipedia.org/wiki/P_versus_NP_problem).  Although we won't be attempting to make a research breakthrough in this question space, it can still be fun and instructive to explore problems like the $n$-queens puzzle.

# General (naïve-ish) solution

First, we'll represent the board in an efficient way: an $n$-dimensional array, where the position reflects the column number and the value reflects the row of the queen at that column.

For example, the above solution to the 8-queens puzzle could be represented by the array [2, 4, 6, 8, 3, 1, 7, 5].

Notice that, with this notation, every possible solution to the $n$-queens puzzle must be a permutation of the integers from 0 to $n-1$. This ensures that at most a single queen is placed in each row and column.

Next we need to check the diagonals. Using the same notation, we can check to see if queens are attacking each other along the "forward" (top-left to bottom-right) diagonal by asking whether any queens share the same difference between their row and column. In other words: if $r_0...(n-1)$ represents the queens' rows and $c_0...(n-1)$ represents the queens' columns, we must ensure that $r_i - c_i$ never equals $r_j - c_j$ for any value of $i$ not equal to $j$. This can be checked efficiently by ensuring that `len(np.unique(np.arange(n) - pos)) == n`. Finally, to check the "reverse" diagonal (i.e. top-right to bottom-left), we can use analogous logic and verify that `len(np.unique(np.arange(n-1, -1, -1) - pos)) == n` as well.

If no queens share a row, column, forward diagonal, or reverse diagonal, and if $n$ queens have been placed on the $n \times n$ board, then we have found a solution!

Naïvely, to find every solution of the $n$-queens puzzle for a given $n$, we could simply iterate through all possible permutations of the integers from $0...(n-1)$, check whether each is a valid solution, and collect up all of the permutations that are valid solutions. The challenge is that the number of permutations of $n$ numbers is $n$ factorial, which becomes intractable as $n$ gets large. For example, storing a single integer in short (16 bit) format requires 2 bytes. Therefore storing a single position (of length $n$) requires $2n$ bytes. Storing all possible $n!$ permutations for a $13 \times 13$ board would require over 12GB of memory, and for a $14 \times 14$ board would require over 174GB!

Since our main concern for this assignment is to improve our programming skills, we won't worry about scalability here; you can assume that you'll never need to solve (or count solutions for) $n > 12$.

# Grading

This assignment is worth a total of 5 points.  You may view the tests in the public rubric [here](https://github.com/ContextLab/cs-for-psych/blob/master/assignments/n-queens/public_rubric.xls).

In [8]:
from sympy.utilities.iterables import multiset_permutations #this is used to compute every permutation of a list

# Represent a board position as a string: `board2str`

We'll write a function to convert a board position to a string that may be printed out:
```python
>> print(board2str([0, 4, 7, 5, 2, 6, 1, 3]))

-|0|1|2|3|4|5|6|7|
0|*|-|-|-|-|-|-|-|
1|-|-|-|-|-|-|*|-|
2|-|-|-|-|*|-|-|-|
3|-|-|-|-|-|-|-|*|
4|-|*|-|-|-|-|-|-|
5|-|-|-|*|-|-|-|-|
6|-|-|-|-|-|*|-|-|
7|-|-|*|-|-|-|-|-|
```

Some things to note:
  - The `board2str` function takes as input a single argument, `pos`: a list representation of the board position
  - From this list representation, the function automatically determines how large the board is (`n = len(pos)`)
  - The rows and columns of the board a numbered (starting with 0)
  - Empty squares are denoted by `-` and queens are denoted by `*`
  - Rows are separated using the [newline character](https://www.freecodecamp.org/news/python-new-line-and-how-to-python-print-without-a-newline/), `\n`.  This tells Python that a single string occupies several lines, e.g. when it is printed out.

Given the input `[0, 4, 7, 5, 2, 6, 1, 3]`, `board2str` should return the following string:
```python
>> board2str([0, 4, 7, 5, 2, 6, 1, 3])

'-|0|1|2|3|4|5|6|7|\n0|*|-|-|-|-|-|-|-|\n1|-|-|-|-|-|-|*|-|\n2|-|-|-|-|*|-|-|-|\n3|-|-|-|-|-|-|-|*|\n4|-|*|-|-|-|-|-|-|\n5|-|-|-|*|-|-|-|-|\n6|-|-|-|-|-|*|-|-|\n7|-|-|*|-|-|-|-|-|'
```
Compare this to the printout above to get a feel for how it works.[link text](https://)

In [4]:
def get_n(pos): #given a board position, compute the board size
  ### BEGIN YOUR CODE
  return 0
  ### END YOUR CODE

def board2str(pos):
  def row2str(c):
    #print out a single row with a queen in column c
    #e.g. row2str(0) should return '|*|-|-|-|-|-|-|-|'

    ### BEGIN YOUR CODE
    return ''
    ### END YOUR CODE
  
  def top_row(n):
    #print out the top row of labels, given the board size
    #e.g. top_row(3) should return '-|0|1|2|'

    ### BEGIN YOUR CODE
    return ''
    ### END YOUR CODE
  
  n = get_n(pos)
  board = [top_row(n)]
  for r in range(n):
    ### BEGIN YOUR CODE
    c = 0 #compute the column of the queen in for row r
    ### END YOUR CODE
    board.append(str(r) + row2st(c))
  
  return '\n'.join(board)

# Check whether a position is a solution: `is_solved`



In [5]:
def is_solved(pos):
  n = get_n(pos)

  #check rows and columns
  ### BEGIN YOUR CODE
  return False
  ### END YOUR CODE

  #check diagonals
  ### BEGIN YOUR CODE
  return False
  ### END YOUR CODE

  #return True if pos is a solution and False otherwise
  ### BEGIN YOUR CODE
  return False
  ### END YOUR CODE

# Solving the puzzle!

Now the fun part-- we can use your work in the cells above to solve the $n$-queens puzzle.  We'll churn through every possible permutation of the numbers $0...(n-1)$, check whether each of these "positions" is a valid solution to the $n$-queens puzzle, and keep track of all of the corresponding permutations (i.e., that checked out as valid solutions).

In [10]:
def get_solutions(n):
    solutions = []
    for p in multiset_permutations(np.arange(n).astype('int16')):
        if is_solved(p):
            solutions.append(p)
    return solutions

# Sanity checks

Let's check that this works for the 8-queens puzzle (there should be 92 solutions)

In [ ]:
eights = get_solutions(8)
assert len(eights) == 92, 'something is wrong...'

And let's also print out one of the solutions; you should get the printout shown below:
```
-|0|1|2|3|4|5|6|7|
0|*|-|-|-|-|-|-|-|
1|-|-|-|-|-|-|*|-|
2|-|-|-|-|*|-|-|-|
3|-|-|-|-|-|-|-|*|
4|-|*|-|-|-|-|-|-|
5|-|-|-|*|-|-|-|-|
6|-|-|-|-|-|*|-|-|
7|-|-|*|-|-|-|-|-|
```

In [ ]:
print(board2str(eights[0]))